In [24]:
import pandas as pd 
import matplotlib.pyplot as plt 
import datetime
import ast
from datetime import timedelta
import subprocess
import sys
import traceback

In [25]:
def extract_sensor_data(df_log):
    l_sensor_data = []
    for row in df_log["sensor_data"]:
        dico = ast.literal_eval(row)
        l_sensor_data.append(dico)
    df_sensor_data = pd.DataFrame(l_sensor_data, 
                                columns=["temp","pressure","vibrations"])
    df = pd.concat([df_log,df_sensor_data],axis=1)
    return df #extraction des données des censeurs

def conversion_str_to_num(df_log):
    try:
        df_log["temp"] = df_log["temp"].str.replace("°C","").astype("float")
        df_log["pressure"] = df_log["pressure"].str.replace("hPa","").astype("float")
        df_log["vibrations_num"] = df_log["vibrations"].str.replace("m/s²","").astype("float")
    except:
        df_log["temp"] = df_log["temp"].str.replace("Â°C","").astype("float")
        df_log["pressure"] = df_log["pressure"].str.replace("hPa","").astype("float")
        df_log["vibrations_num"] = df_log["vibrations"].str.replace("m/sÂ²","").astype("float")
    return df_log #converstion des colonnes 

def conversion_date(df_log):
    df_log["jour_vol"] = pd.to_datetime(df_log["jour_vol"], format=format_date)
    return df_log #conversion en date
    
def avions(df):
    df["end_maint"] = df["end_maint"].fillna("Pas en maintenance")
    df["end_maint"] = df["end_maint"].str.replace(" 14:13:00", "")
    df["last_maint"] = df["last_maint"].str.replace(" 00:00:00", "")
    
    df["last_maint"] = pd.to_datetime(df["last_maint"])
    df["debut_service"] = pd.to_datetime(df["debut_service"])
    return df #conversion en date
    
def cout(df):
    df["cout"] = 0
    df.loc[df["etat_voyant"] == 3, "cout"] = 15000 * 14
    df.loc[df["etat_voyant"] == 2, "cout"] = 15000 * 7
    df.loc[df["etat_voyant"] == 1, "cout"] = 15000
    
    return df  #calcul du coup

In [26]:
base_url = 'http://sc-e.fr/docs/logs_vols_'
format_date = "%Y-%m-%d"
df_avions = pd.read_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\aeronefs_2024-06-02.csv")
date_dep = datetime.datetime.strptime(datetime.datetime.now().strftime(format_date),format_date)
date_fin = datetime.datetime.strptime("2024-06-02", format_date)
date_dep_str = date_dep.strftime(format_date)
date_fin_str = date_fin.strftime(format_date)

log_vol_0206 = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichiers_telecharges\logs_vols_{date_fin_str}.csv")
df_global_maint = pd.DataFrame()
df_global_degra = pd.DataFrame()
df_global_omega = pd.DataFrame()
while date_dep >= date_fin :  # pour chaque date entre le 02 et aujourd'hui
    date_dep_str = date_dep.strftime(format_date)
    try :
        log_vol = fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichiers_telecharges\logs_vols_{date_dep_str}.csv"
        #extraction des données de vol
        data_logs = pd.read_csv(log_vol)
        
        data_logs = data_logs.merge(df_avions, right_on="ref_aero", left_on="aero_linked") #lien avec les aeronefs
        data_logs_good = conversion_date(data_logs)
        data_logs_good = extract_sensor_data(data_logs)
        data_logs_good = conversion_str_to_num(data_logs_good)  
        data_logs_good = avions(data_logs_good)
        data_logs_good = data_logs_good[["jour_vol","ref_vol","aero_linked","time_en_air","etat_voyant",
                            "type_model","debut_service","last_maint","en_maintenance",
                            "end_maint","temp","pressure","vibrations","vibrations_num"]] #filtre sur les colonnes
    

        data_logs_good.loc[data_logs_good["etat_voyant"] != 0, "en_maintenance"] = True
        data_logs_good.loc[data_logs_good["etat_voyant"] == 1, "end_maint"] = pd.to_datetime(date_dep + timedelta(days=1))
        data_logs_good.loc[data_logs_good["etat_voyant"] == 2, "end_maint"] = pd.to_datetime(date_dep + timedelta(days=7))
        data_logs_good.loc[data_logs_good["etat_voyant"] == 3, "end_maint"] = pd.to_datetime(date_dep + timedelta(days=14))
        #calcul du temps de maintenance

        data_logs_good = cout(data_logs_good) #calcul du cout

        data_logs_good.to_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\maint_journa\maintenance_{date_dep_str}.csv")
        #sauvegarde fichier journa
        df_global_maint = pd.concat([data_logs_good, df_global_maint])
        #MAJ df global log
        print(f"{date_dep_str} OK log_vol")
    

    except Exception as e:
        print(e)
        traceback.print_exc()

    try :     
        degrad  = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichiers_telecharges\degradations_{date_dep_str}.csv")
        #import csv du jour des degradations
        compo = pd.read_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\donnees_depart\composants_2024-06-02.csv")
        #import fichier d'origine
        compo = compo[["ref_compo","taux_usure_actuel","cout"]]
        #filtre sur les colonnes du fichier d'origine
        compo.columns = ["ref_compo","taux_usure_dep","cout_compo"]
        #renomme les colonnes
        degrad =  degrad[degrad["measure_day"]==date_dep_str]
        
        data_degrad =  degrad.merge(compo, left_on="compo_concerned", right_on="ref_compo", how="left")
        #merge des dégradations du jour et fichier d'origine (prend l'id le cout du compo)
        data_degrad["usure_totale"] =abs(data_degrad["taux_usure_dep"]-data_degrad["usure_nouvelle"])
        data_degrad.to_csv(fr"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\compo_maj_journa\compo_maj_{date_dep_str}.csv")
        #calcul de l'usure journalière du composant
        df_global_degra = pd.concat([df_global_degra,data_degrad],axis=0)
        #concaténation de chaque fichier dans un df global
        print(f"{date_dep_str} OK degrad")

    except Exception as e1:
        print(e1)
        traceback.print_exc()

    try:
        df_global_omega = df_global_degra.merge(data_logs_good, right_on="aero_linked", left_on="linked_aero",how="left")
        #merge global des dégradations du jours et logs_vols du jour
        print(f"fusion jour {date_dep_str} ok")
    except Exception as e2:
        print(e2)
        traceback.print_exc()
        
    date_dep = date_dep - datetime.timedelta(days=1)
df_global_maint.to_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichier_global\maint_global_history.csv")
#sauvegarde fichier global

2024-06-27 OK log_vol
2024-06-27 OK degrad
fusion jour 2024-06-27 ok
2024-06-26 OK log_vol
2024-06-26 OK degrad
fusion jour 2024-06-26 ok
2024-06-25 OK log_vol
2024-06-25 OK degrad
fusion jour 2024-06-25 ok
2024-06-24 OK log_vol
2024-06-24 OK degrad
fusion jour 2024-06-24 ok
2024-06-23 OK log_vol
2024-06-23 OK degrad
fusion jour 2024-06-23 ok
2024-06-22 OK log_vol
2024-06-22 OK degrad
fusion jour 2024-06-22 ok
2024-06-21 OK log_vol
2024-06-21 OK degrad
fusion jour 2024-06-21 ok
2024-06-20 OK log_vol
2024-06-20 OK degrad
fusion jour 2024-06-20 ok
2024-06-19 OK log_vol
2024-06-19 OK degrad
fusion jour 2024-06-19 ok
2024-06-18 OK log_vol
2024-06-18 OK degrad
fusion jour 2024-06-18 ok
2024-06-17 OK log_vol
2024-06-17 OK degrad
fusion jour 2024-06-17 ok
2024-06-16 OK log_vol
2024-06-16 OK degrad
fusion jour 2024-06-16 ok
2024-06-15 OK log_vol
2024-06-15 OK degrad
fusion jour 2024-06-15 ok
2024-06-14 OK log_vol
2024-06-14 OK degrad
fusion jour 2024-06-14 ok
2024-06-13 OK log_vol
2024-06-13 O

In [27]:
df_global_maint.sample(15)
#sample des logs vols, RAS

,jour_vol,ref_vol,aero_linked,time_en_air,etat_voyant,type_model,debut_service,last_maint,en_maintenance,end_maint,temp,pressure,vibrations,vibrations_num,cout
24,2024-06-03,V06155486,E175_2945,7.1,0,E175,2008-05-15,2024-06-01,False,Pas en maintenance,1.6,882.4,2.082328414130601 m/s²,2.082328,0
66,2024-06-15,V02248515,CRJ700_1002,1.4,1,CRJ700,2007-09-22,2024-02-29,True,2024-06-16 00:00:00,-17.8,889.4,2.3248730160615025 m/s²,2.324873,15000
4,2024-06-23,V01823539,A320_6678,5.7,0,A320,2009-08-24,2024-05-31,False,Pas en maintenance,4.2,863.8,4.976051011188498 m/s²,4.976051,0
20,2024-06-06,V09136742,A330_2237,3.5,0,A330,2019-09-18,2024-06-01,False,Pas en maintenance,-1.2,937.9,2.648864993774333 m/s²,2.648865,0
33,2024-06-10,V05979198,B767_2417,4.9,0,B767,2005-12-31,2024-06-01,False,Pas en maintenance,-1.0,1092.2,1.2735144079828435 m/s²,1.273514,0
39,2024-06-08,V02141751,A350_4789,1.5,0,A350,2022-08-17,2024-05-31,False,Pas en maintenance,-9.0,996.3,1.3184153446919056 m/s²,1.318415,0
56,2024-06-23,V05750301,B737_1936,6.8,0,B737,2008-09-10,2024-06-01,False,Pas en maintenance,18.0,999.9,1.287389357372159 m/s²,1.287389,0
28,2024-06-22,V01087060,CRJ700_4386,7.8,0,CRJ700,2001-09-19,2024-05-31,False,Pas en maintenance,12.1,1022.1,0.18281103862871573 m/sÂ²,0.182811,0
56,2024-06-25,V05173433,A320_6814,4.9,0,A320,2020-02-23,2024-03-14,False,Pas en maintenance,11.6,942.2,4.950755283174505 m/s²,4.950755,0
18,2024-06-10,V08822386,A330_5998,0.6,0,A330,2010-11-15,2024-06-01,False,Pas en maintenance,11.5,961.7,1.5627008207362696 m/s²,1.562701,0


In [28]:
df_global_degra.sample(15)
#sample des dégradations RAS

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement,ref_compo,taux_usure_dep,cout_compo,usure_totale
4160,D002297,A321_0181,AILA321-A321_0181-20,31.588200,2024-06-19,False,AILA321-A321_0181-20,26.947116,10290,4.641084
3892,D003344,CRJ700_0969,FLACRJ700-CRJ700_0969-22,55.038825,2024-06-25,False,FLACRJ700-CRJ700_0969-22,48.434850,14480,6.603975
2536,D002835,E170_3927,GENE170-E170_3927-16,20.884404,2024-06-23,False,GENE170-E170_3927-16,14.753352,10833,6.131052
4377,D004690,A330_5998,SYSA330-A330_5998-12,53.055624,2024-06-10,False,SYSA330-A330_5998-12,52.087587,16266,0.968037
2335,D002822,B757_2518,SYSB757-B757_2518-40,7.518488,2024-06-23,False,SYSB757-B757_2518-40,5.354180,4745,2.164308
3199,D005499,CRJ700_3181,AUTCRJ700-CRJ700_3181-4,12.139963,2024-06-03,False,AUTCRJ700-CRJ700_3181-4,11.308531,19362,0.831432
481,D002428,B777_4962,SIEB777-B777_4962-31,53.326040,2024-06-13,False,SIEB777-B777_4962-31,50.735356,2151,2.590684
4971,D007567,A330_6979,EMPA330-A330_6979-21,48.470712,2024-06-05,False,EMPA330-A330_6979-21,46.451520,10528,2.019192
2090,D007118,A340_1384,AILA340-A340_1384-20,28.508420,2024-06-17,False,AILA340-A340_1384-20,24.667160,10290,3.841260
373,D008611,B757_6806,SYSB757-B757_6806-13,19.529328,2024-06-02,False,SYSB757-B757_6806-13,19.529328,15045,0.000000


In [29]:
df_global_omega_filtered = df_global_omega.dropna(subset="jour_vol") 
#je prends uniquement les données des avions qui ont effectué au moins un vol

In [30]:
df_global_omega_filtered["red_flag"] = df_global_omega_filtered["usure_nouvelle"] > 75
#ajout de la colonne red_flag => tout les composants à plus de 75% d'usure doivent être remplacer 
#si une maintenance est prédite

C:\Users\bapti\AppData\Local\Temp\ipykernel_13556\4128434110.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_global_omega_filtered["red_flag"] = df_global_omega_filtered["usure_nouvelle"] > 75


In [32]:
df_global_omega_filtered = df_global_omega_filtered[['ref_deg', 'linked_aero', 'compo_concerned', 'usure_nouvelle',
       'measure_day', 'need_replacement', 'ref_compo', 'taux_usure_dep',
       'cout_compo', 'jour_vol', 'ref_vol', 'time_en_air',
       'etat_voyant', 'type_model', 'debut_service', 'last_maint',
       'en_maintenance', 'end_maint', 'temp', 'pressure', 'vibrations',"usure_totale",
       'vibrations_num', 'cout', 'red_flag']]
#filtre des colonnes

In [33]:
df_global_omega_filtered.sample(25)
#sample du global 

,ref_deg,linked_aero,compo_concerned,usure_nouvelle,measure_day,need_replacement,ref_compo,taux_usure_dep,cout_compo,jour_vol,...,last_maint,en_maintenance,end_maint,temp,pressure,vibrations,usure_totale,vibrations_num,cout,red_flag
79249,D000102,A380_6924,AUTA380-A380_6924-4,55.835422,2024-06-10,False,AUTA380-A380_6924-4,54.024088,19362,2024-06-02,...,2024-06-02,True,2024-06-03 00:00:00,-16.7,987.1,0.44643993815223937 m/s²,1.811334,0.446440,15000.0,False
21926,D006853,A330_3548,UNIA330-A330_3548-11,31.404952,2024-06-23,False,UNIA330-A330_3548-11,26.472400,17834,2024-06-02,...,2024-04-23,False,Pas en maintenance,8.2,905.9,2.1511054212156973 m/s²,4.932552,2.151105,0.0,False
60980,D000880,B747_4007,SYSB747-B747_4007-5,37.638060,2024-06-14,False,SYSB747-B747_4007-5,32.241240,19492,2024-06-02,...,2024-05-31,False,Pas en maintenance,6.7,983.0,3.173507243818179 m/s²,5.396820,3.173507,0.0,False
55659,D002660,A330_0921,PORA330-A330_0921-39,35.368643,2024-06-16,False,PORA330-A330_0921-39,30.778396,2850,2024-06-02,...,2024-05-31,False,Pas en maintenance,-0.9,968.4,3.2486495155908117 m/s²,4.590247,3.248650,0.0,False
114702,D003252,B757_4533,HUBB757-B757_4533-42,65.020284,2024-06-02,False,HUBB757-B757_4533-42,65.020284,3593,2024-06-02,...,2024-05-31,False,Pas en maintenance,-3.1,944.2,0.10393012355558617 m/s²,0.000000,0.103930,0.0,False
105845,D005330,B777_3776,SYSB777-B777_3776-5,11.557320,2024-06-04,False,SYSB777-B777_3776-5,11.368620,19492,2024-06-02,...,2024-03-05,False,Pas en maintenance,-6.0,925.1,2.2291476661216194 m/s²,0.188700,2.229148,0.0,False
111654,D004959,B757_6806,SYSB757-B757_6806-9,42.120912,2024-06-02,False,SYSB757-B757_6806-9,42.120912,18680,2024-06-02,...,2023-11-18,False,Pas en maintenance,11.1,931.3,2.8166257793381355 m/s²,0.000000,2.816626,0.0,False
55827,D003019,A380_5199,SYSA380-A380_5199-27,49.112874,2024-06-16,False,SYSA380-A380_5199-27,44.963166,13233,2024-06-02,...,2023-11-24,False,Pas en maintenance,1.3,991.2,2.6697111799684654 m/s²,4.149708,2.669711,0.0,False
75158,D009214,A350_1889,SYSA350-A350_1889-8,20.832613,2024-06-11,False,SYSA350-A350_1889-8,18.598602,18351,2024-06-02,...,2024-04-08,False,Pas en maintenance,14.9,983.3,3.5073642959334768 m/s²,2.234011,3.507364,0.0,False
112616,D007780,B747_4659,GOUB747-B747_4659-26,19.223294,2024-06-02,False,GOUB747-B747_4659-26,19.223294,12059,2024-06-02,...,2024-02-17,False,Pas en maintenance,-2.9,956.1,1.297831307345917 m/s²,0.000000,1.297831,0.0,False


In [34]:
df_global_omega_filtered.to_csv(r"C:\Users\bapti\Desktop\WCS Python\Projet 3\Projet-3-Skyanalytics\fichier_global\data_SUPERGLOBAL.csv")